In [83]:
from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit.tools.monitor import job_monitor
from azure.quantum.qiskit import AzureQuantumProvider
import qiskit
import math

In [84]:
_bitCache = ''

In [85]:
def set_provider(token = ''):
    global provider
    provider = AzureQuantumProvider (
        resource_id = "/subscriptions/b1d7f7f8-743f-458e-b3a0-3e09734d716d/resourceGroups/aq-hackathons/providers/Microsoft.Quantum/Workspaces/aq-hackathon-01",
        location = "eastus"
    )

In [86]:
def set_backend(b = 'qasm_simulator'):
    global _backend
    global provider
    _backend = provider.get_backend("ionq.simulator")
    _set_circuit(min(_backend.configuration().n_qubits, 10)) #Develop our circuit, knowing the size of the quantum device

In [87]:
def _set_circuit(n):
    global _circuit
    qr = qiskit.QuantumRegister(n) #Implement a quantum register of n qubits
    cr = qiskit.ClassicalRegister(n) #Implement a classical register of n cbits
    _circuit = qiskit.QuantumCircuit(qr, cr) #Include the quantum register and the classical register in the circuit
    # n = 2
    # |0> ---------
    # |0> ---------
    # ============= 
    # =============
    _circuit.h(qr) # Apply Hadamard gate to all the qubits in the quantum register
    # n = 2
    # |0> --H------
    # |0> --H------
    # =============
    # =============
    _circuit.measure(qr,cr) # Measure all the qubits in the quantum register and store them in the classical register
    # n = 2
    # |0> --H--M---
    # |0> --H-----M
    # =============    <--- classical register now stores the measurement result of each qubit
    # =============

In [88]:
def _get_bits_from_counts(counts):
    for k, v in counts.items():
        if v ==1:
            return k

In [89]:
def _request_bits(n):
    global _bitCache
    global _circuit
    iterations = math.ceil(n * 2/_circuit.width())
    for _ in range(iterations):
        job = _backend.run(_circuit, shots=1)
        _bitCache += _get_bits_from_counts(job.result().get_counts())

In [90]:
def _get_bit_string(n):
    global _bitCache
    if len(_bitCache) < n:
        _request_bits(n-len(_bitCache)) #request as many bits as we are lacking
    bitString = _bitCache[0:n] #result is the first n bits of the bitCache
    _bitCache = _bitCache[n:] #delete the first n bits of the bitCache
    return bitString

In [91]:
def get_collapsed_piece(n): # there is a n/8 chance that the piece collapses to left
    n = n % 8
    measure = _get_bit_string(3)
    return measure >= n # 0 is left and 1 is right

In [92]:
set_provider()
set_backend()
get_collapsed_piece(6)

............

TypeError: can only concatenate str (not "NoneType") to str